In [1]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')
!ls

Mounted at /content/drive
drive  sample_data


In [2]:
# move into project directory
repo_name = "Image-Colorization"
%cd /content/drive/MyDrive/Personal-Projects/$repo_name
!ls

/content/drive/MyDrive/Personal-Projects/Image-Colorization
common	     dataloading  Index.ipynb  output		     README.md	       transforms
config.yaml  experiments  index.py     preprocess	     requirements.txt  visualization
data	     Index_bc.py  models       project-structure.md  run.yaml


In [3]:
# set up environment
# comment out if not required
'''
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install matplotlib numpy pandas pyyaml opencv-python
'''

'\n!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118\n!pip install matplotlib numpy pandas pyyaml opencv-python\n'

In [4]:
# this cell is for downloading data.
# as of yet data is not hosted and is available in the private data folder


In [5]:
# setup some imports
#custom imports
from transforms.transforms import ToTensor
from dataloading.datareader import DataReader
from dataloading.dataset import CustomDataset
from common.utils import get_exp_params, init_config, get_config, save2config, get_saved_model, get_modelinfo
from models.unet import UNet
from models.custom_models import get_model

#py imports
import random
import numpy as np
import os
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from experiments.experiments import Experiment
from visualization.visualization import Visualization
from experiments.test_model import ModelTester

In [ ]:
'''
import torch.nn as nn
import torch

class Encoder(nn.Module):

    def __block(self, in_c, out_c, mp_size = 2, dp = 0.2):
        return nn.Sequential(
            nn.Conv2d(in_c, out_c, kernel_size = 5),
            nn.BatchNorm2d(out_c),
            nn.ReLU(),
            #nn.MaxPool2d(mp_size),
            nn.Dropout(dp)
        )

    def __init__(self):
        super().__init__()
        self.block1 = self.__block(1, 8)
        self.block2 = self.__block(8, 16)
        self.block3 = self.__block(16, 32)
        self.block4 = self.__block(32, 64)

    def forward(self, x):
        print('inp', x.size())
        h1 = self.block1(x) #32 -> 28
        print('h1', h1.size())
        h2 = self.block2(h1) #28 -> 24
        print('h2', h2.size())
        h3 = self.block3(h2) #24 -> 20
        print('h3', h3.size())
        h4 = self.block4(h3) #20 -> 16

        return [x, h1, h2, h3, h4]

class Decoder(nn.Module):

    def __conv_block(self, in_c, out_c):
        return nn.Sequential(
            nn.Conv2d(in_channels = in_c, out_channels = out_c, kernel_size = 3, stride = 1, padding = 1, bias = False),
            nn.BatchNorm2d(out_c),
            nn.ReLU(),
            nn.Dropout(0.1)
        )

    def __block(self, in_c, out_c, kernel_size, stride, padding, out = False):
        return nn.Sequential(
            nn.ConvTranspose2d(in_c, out_c, kernel_size = kernel_size, stride = stride, padding = padding, bias = False),
            nn.BatchNorm2d(out_c),
            nn.LeakyReLU() if not out else nn.Sigmoid(),
            nn.Dropout(0.2) if not out else nn.Identity(),
        )

    def __init__(self):
        super().__init__()
        self.block1 = self.__block(64, 32, 5, 1, 0)
        self.block2 = self.__block(32, 16, 5, 1, 0)
        self.block3 = self.__block(16, 8, 5, 1, 0)
        self.block4 = self.__block(8, 2, 5, 1, 0, True)

        self.cb1 = self.__conv_block(64, 32)
        self.cb2 = self.__conv_block(32, 16)
        self.cb3 = self.__conv_block(16, 8)

    def forward(self, h):
        print('inp', h[-1].size()) #16 x 16
        x = self.block1(h[-1]) #16 -> 32
        print('o1', x.size())

        print('c1', x.size(), h[-2].size())
        x = torch.concat([x, h[-2]], dim=1) #16
        print('i1', x.size())
        x = self.cb1(x)
        print('o2', x.size())
        x = self.block2(x) #16 -> 15
        print('o3', x.size())

        print('c2', x.size(), h[-3].size())
        x = torch.concat([x, h[-3]], dim=1) #8+8 = 16
        print('i2', x.size())
        x = self.cb2(x) #16 -> 8
        print('o2', x.size())
        x = self.block3(x) #8 -> 4
        print('o3', x.size())

        print('c3', x.size(), h[-4].size())
        x = torch.concat([x, h[-4]], dim=1) #4+4 = 8
        print('i3', x.size())
        x = self.cb3(x) #8 -> 4
        print('o4', x.size())
        x = self.block4(x) #4 -> 1
        print('o5', x.size())
        return x

class UNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, ip):
        h = self.encoder(ip)
        op = self.decoder(h)
        return op
'''


"\nimport torch.nn as nn\nimport torch\n\nclass Encoder(nn.Module):\n\n    def __block(self, in_c, out_c, mp_size = 2, dp = 0.2):\n        return nn.Sequential(\n            nn.Conv2d(in_c, out_c, kernel_size = 5),\n            nn.BatchNorm2d(out_c),\n            nn.ReLU(),\n            #nn.MaxPool2d(mp_size),\n            nn.Dropout(dp)\n        )\n\n    def __init__(self):\n        super().__init__()\n        self.block1 = self.__block(1, 8)\n        self.block2 = self.__block(8, 16)\n        self.block3 = self.__block(16, 32)\n        self.block4 = self.__block(32, 64)\n\n    def forward(self, x):\n        print('inp', x.size())\n        h1 = self.block1(x) #32 -> 28\n        print('h1', h1.size())\n        h2 = self.block2(h1) #28 -> 24\n        print('h2', h2.size())\n        h3 = self.block3(h2) #24 -> 20\n        print('h3', h3.size())\n        h4 = self.block4(h3) #20 -> 16\n\n        return [x, h1, h2, h3, h4]\n\nclass Decoder(nn.Module):\n\n    def __conv_block(self, in_c, ou

In [ ]:
# initialize directories and config data
init_config()
config = get_config()
print('Config parameters\n')
print(config)

Config parameters

{'X_key': 'L', 'data_dir': '/content/drive/MyDrive/Personal-Projects/Image-Colorization/data', 'output_dir': '/content/drive/MyDrive/Personal-Projects/Image-Colorization/output', 'root_dir': '/content/drive/MyDrive/Personal-Projects/Image-Colorization', 'use_gpu': True, 'y_key': 'AB'}


In [ ]:
# read experiment parameters
exp_params = get_exp_params()
print('Experiment parameters\n')
print(exp_params)

Experiment parameters

{'transform': {'resize_dim': 256, 'crop_dim': 224}, 'train': {'shuffle_data': True, 'batch_size': 256, 'val_split_method': 'fix-split', 'k': 5, 'val_percentage': 20, 'loss': 'mse', 'batch_interval': 512, 'epoch_interval': 1, 'num_epochs': 100}, 'model': {'name': 'unet', 'optimizer': 'Adam', 'lr': 0.0001, 'weight_decay': 1e-05, 'amsgrad': False, 'momentum': 0.85}, 'test_model': False}


In [ ]:
#initialize randomness seed
seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
#save X_key and y_key
save2config('X_key', 'L')
save2config('y_key', 'AB')

#preprocess data or load preprocessed data
dr = DataReader()
ds = dr.get_split_data()
Ltr, ABtr, ftr_len = ds['Ltr'], ds['ABtr'], ds['ftr_len']
Lte, ABte, te_len = ds['Lte'], ds['ABte'], ds['te_len']
print('Shape of X and y:', ds['Ltr'].shape, ds['ABtr'].shape)
print('Full Train data length:', ftr_len)
print('Test data length:', te_len, '\n')

#transform data
composed_transforms =  transforms.Compose([
    ToTensor(True)
])

#convert to dataset
ftr_dataset = CustomDataset(Ltr, ABtr, ftr_len, composed_transforms)
te_dataset = CustomDataset(Lte, ABte, te_len, composed_transforms)


Shape of X and y: (50000, 1, 32, 32) (50000, 2, 32, 32)
Full Train data length: 50000
Test data length: 10 



In [ ]:
#model training

exp = Experiment(exp_params["model"]["name"], ftr_dataset)
model_history = exp.train()
print("\nModel validation results")

# get best model
model = get_model(exp_params["model"]["name"])
model = get_saved_model(model, '')
model_info = get_modelinfo('')

#visualization results
vis = Visualization(model_info, model_history)
vis.get_results()


Running straight split
	Running Epoch 0
		Running through train dataset
		Running through validation dataset
	Epoch 0 Training Loss: 0.03495370083255693
	Epoch 0 Validation Loss: 0.027948818542063235
	Epoch 0 Validation Accuracy: 0.0

	Running Epoch 1
		Running through train dataset
		Running through validation dataset
	Epoch 1 Training Loss: 0.030224975384771823
	Epoch 1 Validation Loss: 0.02697502397932112
	Epoch 1 Validation Accuracy: 0.0

	Running Epoch 2
		Running through train dataset
		Running through validation dataset
	Epoch 2 Training Loss: 0.02859798292629421
	Epoch 2 Validation Loss: 0.025050485879182814
	Epoch 2 Validation Accuracy: 0.0

	Running Epoch 3
		Running through train dataset
		Running through validation dataset
	Epoch 3 Training Loss: 0.0276014442788437
	Epoch 3 Validation Loss: 0.02482847945066169
	Epoch 3 Validation Accuracy: 0.0

	Running Epoch 4
		Running through train dataset
		Running through validation dataset
	Epoch 4 Training Loss: 0.026896705728722735


KeyboardInterrupt: 

In [ ]:
#model testing
print("\n\nTesting Saved Model")
mt = ModelTester(model, te_dataset)
mt.test_and_plot(ds["RGBte"], "best_model", True)